In [ ]:
# @title # Instalar
import os
import subprocess
from google.colab import drive
drive.mount('/content/drive')
!sudo apt install aria2 -y
!aria2c --console-log-level=error -x 16 -s 16 -k 1M https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/ensemble.py -d /content -o ensemble.py
!git clone https://github.com/ZFTurbo/Music-Source-Separation-Training.git
!mkdir 'Music-Source-Separation-Training/ckpts'
!rm Music-Source-Separation-Training/inference.py
!aria2c --console-log-level=error -x 16 -s 16 -k 1M https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/Music-Source-Separation/inference.py -d /content/Music-Source-Separation-Training -o inference.py
!aria2c --console-log-level=error -x 16 -s 16 -k 1M https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml -d /content/Music-Source-Separation-Training/ckpts -o model_bs_roformer_ep_317_sdr_12.9755.yaml
!aria2c --console-log-level=error -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt -d /content/Music-Source-Separation-Training/ckpts -o model_bs_roformer_ep_317_sdr_12.9755.ckpt
!pip install "audio-separator[gpu]" yt-dlp pydub soundfile &> /dev/null
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/ &> /dev/null

In [ ]:
# @title # Criar Dataset
# Link da música/playlist
# @markdown ## Link da Playlist
link = "" #@param {type:"string"}

import os
import zipfile
from glob import glob
from pydub import AudioSegment
from pathlib import Path
from pydub.silence import split_on_silence

pasta_arquivos_originais = "/content/arquivos_originais"
pasta_arquivos_processados = "/content/arquivos_processados"
pasta_arquivos_vocais = "/content/arquivos_processados/arquivos_vocais"
pasta_arquivos_instrumentais = "/content/arquivos_processados/arquivos_instrumentais"
pasta_arquivos_karaoke = "/content/arquivos_processados/arquivos_karaoke"
pasta_arquivos_dereverb = "/content/arquivos_processados/arquivos_dereverb"
pasta_arquivos_deecho = "/content/arquivos_processados/arquivos_deecho"
pasta_arquivos_denoise = "/content/arquivos_processados/arquivos_denoise"
pasta_arquivos_finais = "/content/arquivos_finais"

folders = [pasta_arquivos_originais, pasta_arquivos_processados, pasta_arquivos_vocais, pasta_arquivos_instrumentais, pasta_arquivos_karaoke, pasta_arquivos_dereverb, pasta_arquivos_deecho, pasta_arquivos_denoise, pasta_arquivos_finais]

for folder in folders:
    path = Path(folder)
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)
    elif path.is_file():
        print(f"{folder} is a file, not a directory.")

def listar_arquivos(pasta):
    return os.listdir(pasta)

def buscar_arquivo_com_palavra(pasta, palavra):
    arquivos = os.listdir(pasta)
    arquivos_com_palavra = [arquivo for arquivo in arquivos if palavra in arquivo]
    if arquivos_com_palavra:
        return arquivos_com_palavra[-1]
    else:
        return None

def ultimo_arquivo_modificado(pasta):
    arquivos = [f for f in os.listdir(pasta) if os.path.isfile(os.path.join(pasta, f))]
    mais_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta, x)))
    return mais_recente

def remover_palavra_nome_arquivo(arquivo, palavra):
    nome_arquivo = os.path.basename(arquivo)
    if palavra in nome_arquivo:
        novo_nome = nome_arquivo.replace(palavra, '')
        novo_arquivo = os.path.join(os.path.dirname(arquivo), novo_nome)
        os.rename(arquivo, novo_arquivo)
        return novo_nome
    return nome_arquivo

print("Baixando música(s)")
!yt-dlp -x -o "{pasta_arquivos_originais}/%(title)s.%(ext)s" "{link}" &> /dev/null
print("Download concluido!")
arquivos_originais = listar_arquivos("/content/arquivos_originais")

for arquivo_original in arquivos_originais:
    # Separa Vocais/Instrumentais
    print("Começando a separar os vocais")
    !python /content/Music-Source-Separation-Training/inference.py --model_type "bs_roformer" --config_path "/content/Music-Source-Separation-Training/ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml" --start_check_point "/content/Music-Source-Separation-Training/ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt" --input_file "{pasta_arquivos_originais}/{arquivo_original}" --store_dir "{pasta_arquivos_vocais}" --extract_instrumental
    arquivo_original_name = os.path.splitext(os.path.basename(f"{pasta_arquivos_originais}/{arquivo_original}"))[0]
    arquivo_voc = buscar_arquivo_com_palavra(pasta_arquivos_vocais, f"{arquivo_original_name}._vocals")
    !audio-separator "{pasta_arquivos_originais}/{arquivo_original}" --log_level warning --single_stem Vocals --normalization 1.0 --mdxc_segment_size 512 --mdxc_overlap 4 -m MDX23C-8KFFT-InstVoc_HQ.ckpt --output_dir "{pasta_arquivos_vocais}"
    arquivo_mdx23c = buscar_arquivo_com_palavra(pasta_arquivos_vocais, f"{arquivo_original_name}_(Vocals)_MDX23C")
    !python ensemble.py --files "{pasta_arquivos_vocais}/{arquivo_voc}" "{pasta_arquivos_vocais}/{arquivo_mdx23c}" --type avg_fft --weights 2 1 --output "{pasta_arquivos_vocais}/{arquivo_original_name}_ensemble.wav"
    arquivo_voc = ultimo_arquivo_modificado(pasta_arquivos_vocais)
    print("Vocais separados!")

    # Dereverb
    print("Começando a tirar o Reverb")
    !audio-separator "{pasta_arquivos_vocais}/{arquivo_voc}" --log_level warning --single_stem Instrumental  --normalization 1.0 -m Reverb_HQ_By_FoxJoy.onnx --output_dir '{pasta_arquivos_dereverb}'
    arquivo_rev = buscar_arquivo_com_palavra(pasta_arquivos_dereverb, f"{arquivo_original_name}_ensemble_(Instrumental)")
    arquivo_rev = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_dereverb/{arquivo_rev}", "_Reverb_HQ_By_FoxJoy")
    print("Reverb removido!")

    # Deecho
    print("Começando a tirar o Echo")
    !audio-separator "{pasta_arquivos_dereverb}/{arquivo_rev}" --log_level warning --normalization 1.0 -m UVR-De-Echo-Aggressive.pth --output_dir '{pasta_arquivos_deecho}'
    arquivo_no_echo = buscar_arquivo_com_palavra(pasta_arquivos_deecho, f"{arquivo_original_name}_ensemble_(Instrumental)_(No Echo)")
    arquivo_no_echo = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_deecho/{arquivo_no_echo}", "_UVR-De-Echo-Aggressive")
    arquivo_echo = buscar_arquivo_com_palavra(pasta_arquivos_deecho, "(Instrumental)_(Instrumental)")
    !rm "/content/arquivos_processados/arquivos_deecho/{arquivo_echo}"
    print("Echo removido!")

    # Karaoke
    print("Começando a tirar os Backing Vocals")
    !audio-separator "{pasta_arquivos_deecho}/{arquivo_no_echo}" --log_level warning --single_stem Instrumental  --normalization 1.0 -m UVR-BVE-4B_SN-44100-1.pth --output_dir '{pasta_arquivos_karaoke}'
    arquivo_karaoke = buscar_arquivo_com_palavra(pasta_arquivos_karaoke, f"{arquivo_original_name}_ensemble_(Instrumental)_(No Echo)_(Instrumental)")
    arquivo_karaoke = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_karaoke/{arquivo_karaoke}", "_UVR-BVE-4B_SN-44100-1")
    print("Backing Vocals removidos!")

    # Denoise
    print("Começando a tirar o Noise")
    !audio-separator "{pasta_arquivos_karaoke}/{arquivo_karaoke}" --log_level warning --single_stem Instrumental --normalization 1.0 -m UVR-DeNoise.pth --output_dir '{pasta_arquivos_denoise}'
    arquivo_denoise = buscar_arquivo_com_palavra(pasta_arquivos_denoise, f"{arquivo_original_name}_ensemble_(Instrumental)_(No Echo)_(Instrumental)_(Instrumental)")
    arquivo_denoise = remover_palavra_nome_arquivo(f"/content/arquivos_processados/arquivos_denoise/{arquivo_denoise}", "_UVR-DeNoise")
    print("Noise removido!")

    # Tirar partes com silencio
    print("Tirando as partes com Silencio")
    arquivo_para_processar = buscar_arquivo_com_palavra(pasta_arquivos_denoise, f"{arquivo_original_name}_ensemble_(Instrumental)_(No Echo)_(Instrumental)_(Instrumental)")
    audio = AudioSegment.from_file(f"/content/arquivos_processados/arquivos_denoise/{arquivo_para_processar}", format="flac")
    chunks = split_on_silence(
        audio,
        min_silence_len=1000,  # Comando para determinar quantos milissegundos definirão uma parte muda
        silence_thresh=-60  # Limite de decibéis para considerar silêncio
    )
    audio_sem_silencio = AudioSegment.empty()
    for chunk in chunks:
        audio_sem_silencio += chunk
    audio_sem_silencio.export(f"{pasta_arquivos_finais}/{arquivo_karaoke}", format="flac")
    print("Partes com silencio removidas!")
    print("Música processada com sucesso!")

origem = '/content/arquivos_finais'
destino = '/content/drive/MyDrive/arquivos_de_voz.zip'

# Criar um arquivo zip no modo de escrita
print("Criando arquivo .zip com arquivos de audio")
with zipfile.ZipFile(destino, 'w') as zipf:
    for root, dirs, files in os.walk(origem):
        for file in files:
            caminho_completo = os.path.join(root, file)
            zipf.write(caminho_completo, os.path.relpath(caminho_completo, origem))
print(f'Arquivo zipado criado em: {destino}')